# Week 6 - Advanced Supervised Learning
Last week, we explored the fundamentals of machine learning, concentrating on model training and evaluation. While this laid the groundwork for constructing an ML pipeline, we did not address methods for enhancing model performance or advanced models for more complex tasks. This week, we will introduce a new model and delve deeper into configuring and evaluating models.

By the end of this module, you should be able to:
1. Explain the components and hyperparameters of a decision tree
2. Explain underfitting and overfitting.
3. Understand the use of validation datasets.
4. Implement a program to tune the hyperparameters of a machine learning model to improve its predictive performance.

### Pima Diabetes Case Study.
We will continue working with the Pima Diabetes Dataset. Below we import the dataset

In [ ]:
from sklearn.datasets import fetch_openml
import pandas as pd
import numpy as np

# we fetch the dataset from https://www.openml.org/search?type=data&status=active&id=37
X,y = fetch_openml(data_id = 37, as_frame = True, return_X_y = True)

# convert tested_positive and tested_negative to 1 and 0
y = (y == 'tested_positive').astype(int)

---

##### **Q1: Split the dataset into a Training and Testing dataset.**

> HINT: Look at Week 5

In [ ]:
from sklearn.model_selection import train_test_split
### Your Code Here.
## NOTE: please use X_train, X_test, y_train, and y_test as your variable names.


---

## Training a Decision Tree
In the pre-module, we introduced the concept of a decision tree and discussed how to interpret it.. Now, let's explore how to train a decision tree using our data. Recall the training procedure for a ML model from Week 5:


1. We input all data points from the dataset \( $x$ \) into the model.
2. The model processes this input, performing computations based on \( $x$ \) and its current weights, to produce predictions, denoted as \( $\hat{y}$ \).
3. We then calculate the error (the opposite of accuracy) for all predictions made by the model.
4. By analyzing the model's correct and incorrect predictions, we adjust the relevant parameters to improve the accuracy of subsequent predictions.
5. We then repeat steps 1-4 until the error ceases to decrease or predefined stopping conditions are met (e.g., we hit a limit on the number of iterations).

Decision Trees are trained in a very similar manner, starting with the root node. The algorithm can be summarized as follows:

1. We input all data points from the dataset \( $x$ \) into the model.
2. Using these datapoints, we create a set of candidate decision rules (`feature <= some value`) by looking at every possible combination of input features and feature values.
3. We evaluate the effectiveness of each candidate rule in classifying the data.
4. 4. The selected rule is then added to our decision tree, forming a new decision node and corresponding leaf nodes.
5. We then repeat steps 1-4 until the error no longer decreases or a stopping condition is met.
6. We assign each leaf node a class based on the majority class that reaches the node.


![fkiw](dtree_flow.png)



In this algorithm, there are two main questions we need to answer:

1. How do we calculate how good a rule is?
2. How do we know when to stop?


### Impurity:
We can measure how good a rule is by measuring the **Impurity** of the resultant groups.

Impurity measures the amount of uncertainty present in the data. Intuitively for a classification problem, we can understand this as "if we were to randomly select a point, how confident are we that we know what the point will be."

Consider the following groups:

![name](groupab.png)


If we were to randomly select a data point from Group A, we have a 2/3 chance of the point being green and a 1/3 chance of it being purple, therefore we can say we are relatively confident that a point chosen from Group A will be green. However, in Group B, there is a 50/50 chance that the point will be green or purple, therefore we are not confident that we can predict the result of randomly choosing a point. In this case, we can say that Group B is less pure or has *higher impurity*

**A group has zero impurity if there is only one class in the group.**


#### Splitting with Impurity

A decision tree utilizes impurity to guide its decision-making process. At each internal node, the tree evaluates the data points that have reached that node and selects a decision rule that creates two subgroups with minimal impurity. Consider the following example classification:

The dataset contains two classes (orange and blue). Based on these points, the decision tree generates several candidate decision rules to partition the space. **The tree then selects the rule that most effectively reduces impurity.**




---
##### **Q2: Below we plot two different candidate splits. Which split will the decision tree choose: Candidate 1 or Candidate 2? Justify your answer**

![splits](splits.png)



*Your Answer Here*

---

### When to Stop

There are various conditions in which we can stop creating new splits in a decision tree, but most of the time we focus on the following four:

1. **Maximum Tree Depth:** The first condition occurs when the newly added node reaches the maximum allowable depth.
2. **Not Enough Samples**: The second condition arises when there are not enough data points to create two splits with a sufficient number of points. For instance, if one of the splits contains only a single sample, the split may not be executed.
3. **No Reduction in Impurity**: TThe third condition is met when the resulting groups from the split are already highly pure, and no further reduction in impurity can be achieved by adding a new split.
4. **No further Splits Possible**: The tree will cease training if all samples are perfectly classified or if the misclassified samples cannot be separated from other samples (e.g., two samples with identical values but different classes).



### Fitting a Decision Tree
Putting everything together, we fit a decision tree onto our training set in the cell below. Like `LogisticRegression`, we don't need to code the Decision Tree training procedure ourselves. Instead, we can simply use the `DecisionTreeClassifier` from `sklearn.tree`


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# create an instance of the Decision Tree model.
dt = DecisionTreeClassifier(
    max_depth = 3, # Condition 1: Stop creating new nodes when we reach a depth of 3
    min_samples_split = 4, # Condition 2: Don't create a new node if there are less than 4 samples in a node.
    min_impurity_decrease = 0.02, # Condition 3: Each node needs to reduce impurity by least 0.05
)

# Fit the model to the training data.
dt.fit(X_train, y_train)

# make predictions
train_predictions = dt.predict(X_train)


Note, while we specify `max_depth,` `min_impurity_decrease` and `min_samples_split` here, if we do not specify a value then the tree will follow condition 4 and keep adding nodes till theres nowhere to go (condition 4).

---

##### **Q3: Use the model to make predictions on the test set. What is the accuracy on the test set?**

In [ ]:
from sklearn.metrics import accuracy_score
### Your Code here

### Make predictions on the test set

### Calculate test accuracy.

---

### Visualizing Our Tree

One benefit of decision trees is that we can visualize the trained tree to gain insights into our data. Below we use the `plot_tree` function from `sklearn.tree` to visualize our model. This function takes in:
1. The trained decision tree
2. The names of the features (`feature_names`)
3. The names of the classes (`class_names`)
4. Flags to control what the visualization outputs including whether to color the boxed (`filled`) and display the impurity of the nodes (`impurity`).

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
# increase figure size
plt.figure(figsize=(10,8))

# run the plot_tree command
plot_tree(dt, # pass in the trained model
          feature_names = X.columns, # provide the feature names
          class_names = ['tested_negative', 'tested_positive'], # provide the class names
          filled = True, # color the nodes
          impurity = False # do not show impurity values (makes tree easier to read)
)
plt.show()

---

## Overfitting and Underfitting
In Week 5, we introduced the concept of splitting the data in order to better evaluate how the model might perform on real-world data. As a reminder, we have two sets of data:

1. Training set: This is the dataset we give the model to learn from. Think of it as homework given to students. The model can practice getting these answers correctly (aka training).
2. Test set: This is data the model has not seen. Think of it as the exam. Student's used homework questions to learn the concepts (training), now we evaluate how well they learned patterns and trends by giving them questions they have not

This week, we will delve more into the reason *why* a model may underperform at a task. Specifically, a model performs poorly in situations when the model is **underfitting** and **overfitting**.

**Underfitting** occurs when a model is too simple to capture the underlying patterns in the data, leading to poor performance on both the training and unseen data. For example, a decision tree can underfit when there are not enough decision nodes since it cannot make enough decisions to adequately distinguish between the classes.

**Overfitting** happens when a model learns the training data too well, including its noise and small details, making it overly complex. As a result, the model performs great on the training data but poorly on unseen data because it fails to capture the broader patterns. For example, if a decision tree is too deep (too many decisions), each point could potentially end up in its own leaf, causing the model to "memorize" the training data instead of generalizing.

### Controlling for Overfitting and Underfitting
So how do we control overfitting and underfitting? Simple: We configure the model's **Hyperparameters**. Hyperparameters are basically the knobs and levers of the model. They are parameters the user sets to control how the model learns. For example, in Logistic Regression, `max_iter` was a hyperparameter controlling the number of training iterations. Similarly, Decision Trees also have hyperparameters you have already seen: `max_depth`, `min_samples_split`, and `min_impurity_decrease`.

* `max_depth`: The first parameter used to control overfitting is `max_depth`. The max_depth specifies the number of sequential decisions that can lead to a leaf that the decision tree model cannot exceed, and no additional decisions will be made even if the model performance is poor. Since every extra depth of the tree doubles the number of leaves, a sufficiently deep tree can easily assign each training sample to its own leaf. This over-complexity has the tendency to fit onto noise instead of actual trends.

* `min_samples_split`: Another way to avoid overfitting is to specify that there must be more than one point in each leaf. The value set for this parameter will determine the number of samples that must remain at the end of each leaf. This will force the tree to focus on patterns that appear multiple times, making it more robust to noise. This also helps very deep trees from assigning 1 leaf per point.

* `min_impurity_decrease`: While the goal of each node is to reduce the impurity in each split, sometimes to promote generalization, we only want our decision tree to consider splits that are "strong" enough, i.e. reduce impurity enough. Therefore setting this parameter will prevent the decision tree model from using any decision rules that lead to a decrease in purity that is less than the set value. This helps in reducing overfitting to noise.



---

##### **Q4: Train a Decision Tree with `max_depth` values between 1 and 4.  Record the train and test accuracy for each tree. At what depth do you think the tree starts to overfit? At what depths does the tree underfit? Justify your answer.**

> HINT: You can use a for loop to iterate over the `max_depth` values.

In [ ]:
## Your Code Here

*Your Answer Here*

---

## Hyperparameter Tuning

While hyperparameters are crucial to a model's performance, we have yet to address the key question: *How do we select the appropriate hyperparameters?*

One might consider testing numerous hyperparameter configurations and choosing the one that yields the best performance on the test set. However, this approach has a fundamental flaw: the test set is intended to represent unseen data. To illustrate, consider the analogy of a machine learning model as a student: the training set samples are homework problems, while the test set is the exam. Selecting hyperparameters based on test set performance is akin to devising a study strategy, taking the exam, receiving the score, adjusting the study strategy, and retaking the same exam repeatedly. Although this may result in a high test score, it provides an overly optimistic assessment that does not accurately reflect the student's true understanding of the subject matter. Similarly, using the test set to determine the best hyperparameters leads to an overly optimistic estimate of the model's real-world performance.

So, what should we do?




### K-Fold Cross-Validation
Well, like studying for the exam, we can use *practice* exams, sets of questions that are similar to the exam (but not the same) to evaluate our performance. We can create these practice exams by randomly selecting homework problems and setting them aside. We can then use these practice exams to evaluate our studying strategy.

In ML, these practice exams are called *validation* data sets and we construct these sets by taking the training dataset, and further randomly split it up into $K$ relatively-equal sized chunks, or "Folds". We will then create a new version of our model, train it using all but one of the folds, and test it on the remaining fold. We will then rotate the folds and repeat this process. At the conclusion, we will average the performance across all folds. This method is known as **K-Fold Cross-Validation.** We will apply this procedure to various hyperparameter configurations

![gscv](grid_search_cross_validation.png)

We can create these folds using the `KFold()` utility from `sklearn.model_selection.` This takes a dataset, assigns samples to different folds, and provides the indices of what samples are in each fold. Below we create 5 Folds.


In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5,
           shuffle = True) # make sure you set shuffle to true
fold_indices = kf.split(X_train)
for fold_index in fold_indices:
    # Get what samples should be in each split
    train_fold_indices, val_fold_indices = fold_index

    # select the samples
    X_train_fold = X_train.iloc[train_fold_indices]
    y_train_fold = y_train.iloc[train_fold_indices]
    X_val_fold = X_train.iloc[val_fold_indices]
    y_val_fold = y_train.iloc[val_fold_indices]

    print("---------------------------------")
    print(f"Number of samples in training fold: {len(train_fold_indices)}")
    print(f"Number of samples in validation fold: {len(val_fold_indices)}")

---

##### **Q5: Below we create the procedure for testing one hyperparameter configuration of a decision tree. Fill in the blank pieces.**

In [ ]:
def test_hyperparameter(X_train, y_train, max_depth, min_samples_split, min_impurity_decrease):
  # create folds
  kf = KFold(n_splits = 5)
  fold_indices = kf.split(X_train)
  accuracies = []
  # loop through the folds
  for fold_index in fold_indices:
    # Get what samples should be in each split
    train_fold_indices, val_fold_indices = fold_index
    # select the samples
    X_train_fold = X_train.iloc[train_fold_indices]
    y_train_fold = y_train.iloc[train_fold_indices]
    X_val_fold = X_train.iloc[val_fold_indices]
    y_val_fold = y_train.iloc[val_fold_indices]


    # initialize a new decision tree
    dt = ... # YOUR CODE HERE

    # Train the decision tree on the train fold

    # YOUR CODE HERE
  
    # Make predictions on the validation fold
    val_predictions  = ... # YOUR CODE HERE

    # calculate the accuracy of the val predictions

    fold_accuracy = ... # YOUR CODE HERE

    accuracies.append(fold_accuracy)
  return np.mean(accuracies) # return the average accuracy across all folds


## Run the function with a set of parameters
cv_acc = test_hyperparameter(X_train, y_train,
                             max_depth = 3,
                             min_samples_split = 3,
                             min_impurity_decrease = 0.01)

print(f"The average cross-validation accuracy is: {cv_acc}")


---

### Parameter Grid Search

Having established the correct method for evaluating hyperparameters, we now need to identify the optimal parameters. This is accomplished through a process known as 'Grid Search,' where we test every possible combination of parameters to determine which configuration yields the best cross-validation score. Below, we test three different values of `max_depth`, `min_samples_split`, and `min_impurity_decrease` each for a total of $3 \times 3 \times 3 = 27$ combinations.  


In [ ]:
for max_depth in [1,2,3]:
  for min_samples_split in [2,3,4]:
    for impurity in [0.00, 0.01, 0.02]:
      cv_acc = test_hyperparameter(X_train, y_train,
                                   max_depth = max_depth,
                                   min_samples_split = min_samples_split,
                                   min_impurity_decrease = impurity)
      print(f"max_depth: {max_depth}, min_samples_split: {min_samples_split}, min_impurity_decrease: {impurity}, cv_acc: {cv_acc:.3f}")

After finding the best set of parameters, we then train the model on *all* the training data and do our final evaluation:

---

##### **Q6: What set of parameters had the best CV accuracy? Train a model using these parameters using the entire training set. Report the training and testing accuracy.**

*Your Answer Here*

In [ ]:
## Your Code Here

---



## Graded Questions


---

##### **GQ1: Read in the heart failures dataset (`hf_data_tut.csv`) (1pt). Split the predictor variables (features) from the labels (response variable) (1pt), and create a training and testing set (1pt). **

> HINT: Look at your work from Week 3 and 5.



In [ ]:
## Your Code Here

import pandas as pd
#import the data set

#assign a portion of the data to the response variable
y = ...

#assign a portion of the data as predictor variables
X = ...

#create traning and testing data subsets



---
##### **GQ2: Train a decision tree of Depth 2 and visualize the tree (1pt). What feature is used in the root node (1pt)? How many leaves does the tree have (1pt)?**

In [ ]:
## Your Code Here
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

# Setup decision tree, specify the depth
dt = ...

# fit your data to the decison tree

# plot your decision tree

# increase figure size

# run the plot_tree command


*Your Answer Here*

---
##### **GQ3: Print the testing and training accuracy of the model above (1pt). Do you think the model is under or overfitting (or neither) and why (2pt)?**

In [ ]:
## Your Code Here

train_pred = ...
test_pred = ...
train_accuracy = ...
test_accuracy = ...
print(f"Train Accuracy: {train_accuracy:.3f}")
print(f"Test Accuracy: {test_accuracy:.3f}")

Train Acc is significantly higher than test accuracy, therefore the model is probably overfitting.

---


## Conclusion


In conclusion, decision trees are powerful and intuitive tools in machine learning, offering both flexibility and interpretability. By iteratively splitting data based on feature values, decision trees can model complex relationships and make predictions for both classification and regression tasks. Additionally, their ability to be visualized and easily interpreted makes them a valuable analytical tool.

However, with this flexibility comes the risk of overfitting, where the model becomes too tailored to the training data and loses its ability to generalize to new data. To prevent this, various parameters can be tested using K-Fold Cross Validation to find the best configuration. With this knowledge, you can confidently apply ML to a wide range of problems, making them a valuable tool in your machine learning toolkit.